# Nikkei High Dividend Yield 50 Analysis with Portfolio Optimizer



## Initialization

If you are using CoLab, first install the dependency PyPortfolioOpt

In [ ]:
!pip install git+https://github.com/robertmartin8/PyPortfolioOpt.git

Then get our PortfolioOptimizer library and the necessary datasets from our repository

In [ ]:
!wget https://github.com/cartasuzuki/phynance/blob/master/datasets/nikkei_high_dividend_yield_50_prices.csv
!wget https://raw.githubusercontent.com/cartasuzuki/phynance/master/datasets/nikkei_high_dividend_yield_50_weight_en.csv
!wget https://raw.githubusercontent.com/cartasuzuki/phynance/master/PortfolioOptimizer.py

Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PortfolioOptimizer import PortfolioOptimizer

In [ ]:
stock_symbols = pd.read_csv('nikkei_high_dividend_yield_50_weight_en.csv')

Once we have loaded our stock symbols dataset we can either use the prices in the csv file or download prices from alphadvantage to get updated data.

### Filter out some stocks from the index

Chose a minimum div/yield. Set to 0 if you want to use all stocks in the index

In [ ]:
min_yield = 3.5

In [ ]:
selected_stocks = stock_symbols[stock_symbols['Dividend']>min_yield]

Create a filter string (highDivString) to be used later to filter stocks with lower yield than min_yield

In [ ]:
highDivString = selected_stocks['Code'].values.astype(int)
highDivString = highDivString.astype(str)
string = '.TOK'
highDivString = [x + string for x in highDivString]

### Method 1: use csv file

Read the stock prices from the csv provided in our repository

In [ ]:
stocks = pd.read_csv('nikkei_high_dividend_yield_50_prices.csv', index_col= ['timestamp'], parse_dates= ['timestamp'])

If you want higher yield filter out using the filter previously created

In [ ]:
stocks = stocks[highDivString]

### Method 2: Get prices from alphadvantage

Using the highDivString filter, download prices from alpadvantage

In [ ]:
stocks = PortfolioOptimizer.get_stock_prices(highDivString)  

## Portfolio Optimization

In [ ]:
weights, sharpe, ret = PortfolioOptimizer.optimize_portfolio(stocks,0)

In [ ]:
PortfolioOptimizer.print_portfolio_result(weights, sharpe, ret)
PortfolioOptimizer.portfolioAsPieChart(weights)

Average yield

In [ ]:
selected_stocks['Dividend'].mean()

Remove 0s and TOK string.

In [ ]:
www ={x:y for x,y in weights.items() if y>0.001}
portfolio =list(www.keys())
portfoliovalues = list(www.values())
portfolio = [s.replace('.TOK','') for s in portfolio]

### Resulting Portfolio

In [ ]:
selected_stocks = selected_stocks[selected_stocks['Code'].isin(portfolio)]
selected_stocks['Weight'] = portfoliovalues

In [ ]:
selected_stocks

In [ ]:
 Div_Yield = np.average(selected_stocks['Dividend'], weights=selected_stocks['Weight'])
round(Div_Yield, 2)

In [ ]:
www1 ={x:y for x,y in weights1.items() if y>0.001}
portfolio1 =list(www1.keys())
portfoliovalues1 = list(www1.values())
portfolio1 = [s.replace('.TOK','') for s in portfolio]

In [ ]:
weights

In [ ]:
selected_stocks1 = selected_stocks[selected_stocks['Code'].isin(portfolio)]
selected_stocks1['Weight'] = portfoliovalues

In [ ]:
selected_stocks1 = selected_stocks[selected_stocks['Code'].isin(portfolio)]

In [ ]:
selected_stocks

In [ ]:
portfolio